In [1]:
import AdventOfCode

In [2]:
data Direction = U | D | L | R deriving (Eq, Show, Ord)

In [3]:
input <- dayLines 3

In [4]:
import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Attoparsec.Combinator as A

In [5]:
dirnum = do
    dir <- A.choice [A.char 'L' *> pure L, A.char 'D' *> pure D, A.char 'R' *> pure R, A.char 'U' *> pure U]
    num <- A.decimal
    pure (dir, num)

directions = parsed (A.sepBy' dirnum (A.char ','))

Line 2: Use $>
Found:
A.char 'L' *> pure L
Why not:
A.char 'L' Data.Functor.$> LLine 2: Use $>
Found:
A.char 'D' *> pure D
Why not:
A.char 'D' Data.Functor.$> DLine 2: Use $>
Found:
A.char 'R' *> pure R
Why not:
A.char 'R' Data.Functor.$> RLine 2: Use $>
Found:
A.char 'U' *> pure U
Why not:
A.char 'U' Data.Functor.$> U

In [6]:
import qualified Data.Set as S

run :: (Int, Int) -> (Direction, Int) -> ([(Int, Int)], (Int, Int))
run (x,y) (d, n) = case d of
    R -> let
        xs = [1..n]
        cs = map (\m -> (x+m, y)) xs
        in (cs, (x+n,y))
    L -> let
        xs = [1..n]
        cs = map (\m -> (x-m, y)) xs
        in (cs, (x-n,y))
    U -> let
        ys = [1..n]
        cs = map (\m -> (x,y+m)) ys
        in (cs, (x,y+n))
    D -> let
        ys = [1..n]
        cs = map (\m -> (x,y-m)) ys
        in (cs, (x,y-n))

In [7]:
import Data.List (foldl')

execute dirs = S.fromList $ fst $ foldl' (\(points, point) dn -> let (ps, pt) = run point dn in (points++ps,pt)) ([],(0,0)) dirs

In [8]:
line1 = execute $ directions (head input)
line2 = execute $ directions (head $ tail input)

In [9]:
S.findMin $ S.intersection line1 line2

(258,0)

In [10]:
import qualified Data.Map.Strict as M

run' :: ((Int, Int), Int) -> (Direction, Int) -> ([((Int, Int), Int)], ((Int, Int), Int))
run' ((x,y),l) (d, n) = case d of
    R -> let
        xs = [1..n]
        cs = map (\m -> ((x+m, y), l+m)) xs
        in (cs, last cs)
    L -> let
        xs = [1..n]
        cs = map (\m -> ((x-m, y), l+m)) xs
        in (cs, last cs)
    U -> let
        ys = [1..n]
        cs = map (\m -> ((x,y+m), l+m)) ys
        in (cs, last cs)
    D -> let
        ys = [1..n]
        cs = map (\m -> ((x,y-m), l+m)) ys
        in (cs, last cs)

execute' dirs = M.fromList $ fst $ foldl' (\(points, point) dn -> let (ps, pt) = run' point dn in (points++ps,pt)) ([],((0,0),0)) dirs

In [11]:
line1' = execute' $ directions (head input)
line2' = execute' $ directions (head $ tail input)

S.findMin $ S.map (\match -> line1' M.! match + line2' M.! match) $ S.intersection (M.keysSet line1') (M.keysSet line2')

12304